In [ ]:
from __future__ import print_function
import os,sys
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import torch
import ROOT as rt
from larcv import larcv
from drawbbox import DrawBBox

In [ ]:
io = larcv.IOManager()
io.add_in_file( "dlprod_192px_00.root" )
io.initialize()

In [ ]:
io.read_entry(0)

In [ ]:
ev_data = io.get_data( larcv.kProductSparseTensor3D, "data")

In [ ]:
meta = ev_data.meta()
print(meta.dump())
bboxdrawer = DrawBBox(meta,draw_voxels=True)

In [ ]:
vox_v = ev_data.as_vector()
print("num voxels: ",vox_v.size())

In [ ]:
npvoxels = np.zeros( (192,192,192) )
pos = np.zeros( (vox_v.size(),4) )
for i in range(vox_v.size()):
    voxel = vox_v.at(i)
    px = meta.id_to_x_index( voxel.id() )
    py = meta.id_to_y_index( voxel.id() )
    pz = meta.id_to_z_index( voxel.id() )
    pos[i,0] = px
    pos[i,1] = py
    pos[i,2] = pz
    pos[i,3] = voxel.value()
    #print(px,py,pz)
    npvoxels[px,py,pz] = voxel.value()

In [ ]:
# PLOT THE DATA
plot = {"type":"scatter3d",
        "x":pos[:,0],
        "y":pos[:,1],
        "z":pos[:,2],
        "mode":"markers",
        "name":"data",
        "marker":{"color":pos[:,3],"size":1,"opacity":0.8,"colorscale":'Viridis'},
    }
plots = [plot] + bboxdrawer.getlines()


# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='In-time track clusters',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 1},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plots, layout=layout)
fig.show()